In [1]:
import random
import torch
from d2l import torch as d2l

In [2]:
def synthetic_data(w, b, num):
    X = torch.normal(0,1,(num, len(w)))
    y = torch.matmul(X,w) + b
    return X,y;


In [3]:
# 生成数据集
true_w = torch.tensor([10.0,5.0])
true_b = -9
features, labels = synthetic_data(true_w, true_b, 1000)
print(features[:3],'\n',labels[:3])

tensor([[ 0.0383,  1.6482],
        [ 0.6794,  1.7488],
        [ 0.3071, -0.5048]]) 
 tensor([-0.3763,  6.5384, -8.4533])


In [4]:
# 读取数据集
def data_iter(batch_size, features, labels):
    num = len(features)
    indices = list(range(num))
    random.shuffle(indices)
    for i in range(0, num, batch_size):
        batch_indices = torch.tensor(indices[i:min(i + batch_size, num)])
        yield features[batch_indices], labels[batch_indices]

In [5]:
batch_size = 10
for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break

tensor([[-7.3671e-01,  1.2930e+00],
        [-5.9924e-01,  5.6844e-01],
        [ 4.3330e-01,  1.7135e+00],
        [-3.6427e-04,  8.6274e-01],
        [-5.4482e-01,  1.1255e+00],
        [-9.9594e-01, -1.6810e+00],
        [-6.8252e-01,  8.7013e-01],
        [ 5.7838e-01,  1.6595e+00],
        [ 1.8500e+00, -2.7463e-01],
        [-1.7059e+00,  9.7535e-01]]) 
 tensor([ -9.9020, -12.1502,   3.9003,  -4.6899,  -8.8208, -27.3641, -11.4745,
          5.0815,   8.1272, -21.1819])


In [6]:
# 定义模型
def linreg(X, w, b):
    return torch.matmul(X, w) + b
# 定义损失函数
def squard_loss(y_hat, y):
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

In [7]:
# 定义优化算法
def sgd(params, lr):
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad
            param.grad.zero_()

In [8]:
# 初始化模型参数
w = torch.normal(0, 0.1, (2,1),requires_grad = True)
b = torch.ones(1, requires_grad = True)
print(f'w{w},b{b}')
# 训练
lr = 1
num_epochs = 5
net = linreg
loss = squard_loss
for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        y_hat = net(X, w, b)
        l = loss(y_hat, y)
        l.mean().backward()
        sgd([w, b], lr)
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch{epoch + 1},loss:{float(train_l.mean()):6f}')

wtensor([[ 0.0231],
        [-0.0560]], requires_grad=True),btensor([1.], requires_grad=True)
epoch1,loss:0.000000
epoch2,loss:0.000000
epoch3,loss:0.000000
epoch4,loss:0.000000
epoch5,loss:0.000000


In [9]:
print('w:',true_w - w.reshape(true_w.shape), w)
print('b:',true_b - b,b)

w: tensor([0., 0.], grad_fn=<SubBackward0>) tensor([[10.],
        [ 5.]], requires_grad=True)
b: tensor([0.], grad_fn=<RsubBackward1>) tensor([-9.], requires_grad=True)
